In [1]:
#create a nodes and edges dataset from the ross data
#each node is a painting and the edges will have a source, target, and weight
#weight will be based on how many features are shared between a painting

In [98]:
import pandas as pd

In [99]:
bob_df = pd.read_csv("bob-ross.csv")

In [100]:
bob_df.head()

,EPISODE,TITLE,APPLE_FRAME,AURORA_BOREALIS,BARN,BEACH,BOAT,BRIDGE,BUILDING,BUSHES,...,TOMB_FRAME,TREE,TREES,TRIPLE_FRAME,WATERFALL,WAVES,WINDMILL,WINDOW_FRAME,WINTER,WOOD_FRAMED
0,S01E01,"""A WALK IN THE WOODS""",0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0
1,S01E02,"""MT. MCKINLEY""",0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
2,S01E03,"""EBONY SUNSET""",0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
3,S01E04,"""WINTER MIST""",0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0
4,S01E05,"""QUIET STREAM""",0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [101]:
#so we're not representing too much data in the network we'll do 200 paintings
bob_df = bob_df.sample(200)
bob_df.head()

,EPISODE,TITLE,APPLE_FRAME,AURORA_BOREALIS,BARN,BEACH,BOAT,BRIDGE,BUILDING,BUSHES,...,TOMB_FRAME,TREE,TREES,TRIPLE_FRAME,WATERFALL,WAVES,WINDMILL,WINDOW_FRAME,WINTER,WOOD_FRAMED
36,S03E11,"""RUSTIC BARN""",0,0,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
347,S27E10,"""SUNLIGHT IN THE SHADOWS""",0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0
170,S14E02,"""MEADOW BROOK SURPRISE""",0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
345,S27E08,"""DAYBREAK""",0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
380,S30E04,"""WILDERNESS TRAIL""",0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0


In [102]:
#let's create the edge csv
edges = pd.DataFrame(columns=['source', 'target', 'score'])
for index, row in bob_df.iterrows():
    source = index
    for i, r in bob_df[index+1:].iterrows():
        target = i
        score = 0
        for col in bob_df.columns[2:]:
            if row[col] == r[col]:
                score = score+  1
        score = score/67
        if score > 0.90:
            edges = edges.append({'source': int(source), 'target':int(target), "score":score}, ignore_index=True)

In [103]:
print(edges.shape)
edges.head()

(1826, 3)


,source,target,score
0,36.0,64.0,0.910448
1,36.0,315.0,0.925373
2,36.0,164.0,0.940299
3,36.0,396.0,0.940299
4,36.0,397.0,0.910448


In [105]:
#data isn't really doing much for us, let's drop links at a certain cutoff
#edges['score'].quantile(.75)
#edges['score'].mean()

0.9300812476500369

In [121]:
#let's make the node data!
nodes = pd.DataFrame(columns=['id', 'name', 'episode', 'season','features'])
for index, row in bob_df.iterrows():
    feature = ''
    for col in bob_df.columns[2:]:
        if row[col] == 1:
            feature = feature + col + ", "
    nodes = nodes.append({'id': float(index), 'name':row['TITLE'], 
                          "episode":row['EPISODE'], "season": int(row['EPISODE'][1:3]),
                         "features":feature}, ignore_index=True)
nodes.head(10)

,id,name,episode,season,features
0,36.0,"""RUSTIC BARN""",S03E11,3,"BARN, DECIDUOUS, FENCE, LAKE, STRUCTURE, TREE,..."
1,347.0,"""SUNLIGHT IN THE SHADOWS""",S27E10,27,"BUSHES, DECIDUOUS, GRASS, GUEST, RIVER, SUN, T..."
2,170.0,"""MEADOW BROOK SURPRISE""",S14E02,14,"CONIFER, GRASS, RIVER, TREE, TREES,"
3,345.0,"""DAYBREAK""",S27E08,27,"CABIN, CONIFER, DECIDUOUS, FENCE, FOG, SNOW, S..."
4,380.0,"""WILDERNESS TRAIL""",S30E04,30,"BUSHES, DECIDUOUS, PATH, TREE, TREES,"
5,223.0,"""MOUNTAIN SECLUSION""",S18E03,18,"CABIN, CLOUDS, CONIFER, MOUNTAIN, MOUNTAINS, R..."
6,171.0,"""MOUNTAIN MOONLIGHT OVAL""",S14E03,14,"CONIFER, DECIDUOUS, FRAMED, GRASS, MOUNTAIN, M..."
7,29.0,"""WINTER NIGHT""",S03E04,3,"CABIN, CONIFER, SNOW, STRUCTURE, TREE, TREES, ..."
8,219.0,"""NATURE'S SPLENDOR""",S17E12,17,"CONIFER, GRASS, MOUNTAIN, RIVER, SNOWY_MOUNTAI..."
9,126.0,"""OCEAN SUNSET""",S10E10,10,"BEACH, CLOUDS, OCEAN, ROCKS, WAVES,"


In [123]:
#save both to csv files
nodes.to_csv('bob_nodes.csv', index=False)

In [108]:
edges.to_csv('bob_edges.csv', index=False)

In [120]:
nodes.shape

(200, 5)